In [16]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages

True

# Data Collection

In [2]:
with open('Chatbot.txt','r', encoding='utf8', errors ='ignore') as fin:
    raw_doc = fin.read().lower()

In [3]:
raw_doc[:500]

'a chatbot (also known as a talkbot, chatterbot, bot, im bot, interactive agent, or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods. such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the turing test. chatbots are typically used in dialog systems for various practical purposes including customer service or information acqu'

# Text Preprocessing

In [17]:
nltk.download('punkt') #using the punkt tokenizer
nltk.download('wordnet') #using wordnet dictionary (lemmetizer)
sent_tokens = nltk.sent_tokenize(raw_doc)
word_tokens = nltk.word_tokenize(raw_doc)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sudip\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sudip\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
sent_tokens[:3]

['a chatbot (also known as a talkbot, chatterbot, bot, im bot, interactive agent, or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods.',
 'such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the turing test.',
 'chatbots are typically used in dialog systems for various practical purposes including customer service or information acquisition.']

In [19]:
word_tokens[:5]

['a', 'chatbot', '(', 'also', 'known']

In [20]:
lemmer = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punc_dict = dict((ord(punct),None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punc_dict)))

# Using Customized Greetings

In [23]:
greet_inputs = ('hey','hello','hi','whassup','how are you?')
greet_responses = ('hi','hello','Hey there !','How can I help you ?')
def greet(sentence):
    for word in sentence.split():
        if word.lower() in greet_inputs:
            return random.choice(greet_responses)
greet('Hi')

'How can I help you ?'

# Response Generation By Bot

In [24]:
def response(user_response, sent_tokens):
    robo_response=''
    sent_tokens.append(user_response)  # Add user response to corpus
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

# Learning mechanism

In [25]:
def learn_from_user_input(user_response, filename='Chatbot.txt'):
    with open(filename, 'a', encoding='utf8') as fout:
        fout.write('\n' + user_response.lower())

# User Interaction

In [26]:
flag = True
print("Hey! I am Learn Bot. Ask me a question or type bye for ending convo!!")
while(flag==True):
    user_response=input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if (user_response =='thank you' or user_response=='thanks'):
            flag=False
            print("Learn Bot: You are Welcome ^_^")
        else:
            if(greet(user_response) != None):
                print('Learn Bot: ',greet(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response, sent_tokens))
                learn_from_user_input(user_response)  # Capture new response to learn
                sent_tokens.remove(user_response)
    else:
        flag = False
        print('Learn Bot: Goodbye')

Hey! I am Learn Bot. Ask me a question or type bye for ending convo!!
hey
Learn Bot:  hi
what is chatbot
ROBO: design
the chatbot design is the process that defines the interaction between the user and the chatbot.the chatbot designer will define the chatbot personality, the questions that will be asked to the users, and the overall interaction.it can be viewed as a subset of the conversational design.
what is talkbot
ROBO: a chatbot (also known as a talkbot, chatterbot, bot, im bot, interactive agent, or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods.
are chatbot and talkbot same?
ROBO: a chatbot (also known as a talkbot, chatterbot, bot, im bot, interactive agent, or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods.
bye
Learn Bot: Goodbye
